In [70]:
import numpy as np
import pandas as pd
import seaborn as sn
from matplotlib.pyplot import plot
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures,OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
# import warnings
from sklearn.compose import ColumnTransformer

In [35]:
df = pd.read_csv("C:/Users/varis/Projects/mlproject/notebook/data/stud.csv")
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [38]:
X = df.drop('math_score', axis=1)
y= df['math_score']

In [40]:
print(X.shape,y.shape)

(1000, 7) (1000,)


In [47]:
print(X['gender'].unique())

['female' 'male']


In [50]:
num_features = X.select_dtypes(exclude = "object").columns
cat_features = X.select_dtypes(include = "object").columns

In [51]:
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
      ("OneHotEncoder" , oh_transformer, cat_features),
      ("StandardScaler", numeric_transformer, num_features)
    ]
)

In [52]:
X = preprocessor.fit_transform(X)


In [53]:
X.shape

(1000, 19)

In [54]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]], shape=(1000, 19))

In [55]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state=42)

In [57]:
print(x_train.shape,x_test.shape)

(800, 19) (200, 19)


In [74]:
def evalaute_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_s = r2_score(true,predicted)
    return mae,rmse,r2_s

In [63]:
models = {
"Linear Regression": LinearRegression(),
"lasso": Lasso(),
"Ridge": Ridge(),
"K-Neighbors Regressor": KNeighborsRegressor(),
"Decision Tree": DecisionTreeRegressor(),
"Random Forest Regressor": RandomForestRegressor(),
"XGB Regressor": XGBRegressor(),
"CatBoost Regressor": CatBoostRegressor(),
"AdaBoost Regressor": AdaBoostRegressor()

}

In [78]:
model_list = []
r2_list = []


for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(x_train,y_train)

    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    model_train_mae, model_train_rmse, model_train_r2 = evalaute_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evalaute_model(y_test, y_test_pred)


    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Model performance for Training  set")
    print(f"- Root Mean Squared error:{model_train_rmse:.4f}")
    print(f"- Mean Abosolute error:{model_train_mae:.4f}")
    print(f"- R2 score:{model_train_r2:.4f}")

    print("-----------------------------------------------------------------")

    print("Model performance for Test  set")
    print(f"- Root Mean Squared error:{model_test_rmse:.4f}")
    print(f"- Mean Abosolute error:{model_test_mae:.4f}")
    print(f"- R2 score:{model_test_r2:.4f}")

    r2_list.append(model_test_r2)


    print("="*35)
    print('\n')

Linear Regression
Model performance for Training  set
- Root Mean Squared error:5.3231
- Mean Abosolute error:4.2667
- R2 score:0.8743
-----------------------------------------------------------------
Model performance for Test  set
- Root Mean Squared error:5.3940
- Mean Abosolute error:4.2148
- R2 score:0.8804


lasso
Model performance for Training  set
- Root Mean Squared error:6.5938
- Mean Abosolute error:5.2063
- R2 score:0.8071
-----------------------------------------------------------------
Model performance for Test  set
- Root Mean Squared error:6.5197
- Mean Abosolute error:5.1579
- R2 score:0.8253


Ridge
Model performance for Training  set
- Root Mean Squared error:5.3233
- Mean Abosolute error:4.2650
- R2 score:0.8743
-----------------------------------------------------------------
Model performance for Test  set
- Root Mean Squared error:5.3904
- Mean Abosolute error:4.2111
- R2 score:0.8806


K-Neighbors Regressor
Model performance for Training  set
- Root Mean Square

In [80]:
pd.DataFrame(list(zip(model_list,r2_list)), columns=['model Nme', 'R2_Score']).sort_values(by=["R2_Score"], ascending = False)

,model Nme,R2_Score
2,Ridge,0.880593
0,Linear Regression,0.880433
8,AdaBoost Regressor,0.851945
7,CatBoost Regressor,0.851632
5,Random Forest Regressor,0.846204
6,XGB Regressor,0.827797
1,lasso,0.825320
3,K-Neighbors Regressor,0.783813
4,Decision Tree,0.726615
